In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pandas as pd

data = pd.read_csv(r'C:\Users\Hp\Desktop\Suicide_Detection.csv')
print(data.head())


   Unnamed: 0                                               text        class
0           2  Ex Wife Threatening SuicideRecently I left my ...      suicide
1           3  Am I weird I don't get affected by compliments...  non-suicide
2           4  Finally 2020 is almost over... So I can never ...  non-suicide
3           8          i need helpjust help me im crying so hard      suicide
4           9  I’m so lostHello, my name is Adam (16) and I’v...      suicide


In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
import findspark
findspark.init()
conf = SparkConf().set("spark.driver.extraJavaOptions", "-Djava.library.path=C:/Program Files/Java/jdk-11.0.12")
spark = SparkSession.builder \
    .appName("JupyterSpark") \
    .getOrCreate()

In [4]:
spark

In [5]:
import pyspark
print(pyspark.__version__)

3.5.3


In [6]:
# Load the CSV file into a Spark DataFrame
df = spark.read.csv(r'C:\Users\Hp\Desktop\Suicide_Detection.csv', header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()


+--------------------+--------------------+--------------------+
|                 _c0|                text|               class|
+--------------------+--------------------+--------------------+
|                   2|Ex Wife Threateni...|             suicide|
|                   3|Am I weird I don'...|         non-suicide|
|                   4|"Finally 2020 is ...|         non-suicide|
|                   8|i need helpjust h...|             suicide|
|                   9|I’m so lostHello,...|                NULL|
|I was later trans...|                NULL|                NULL|
|Come to find out ...| my mom has been ...| therapy has help...|
|I eventually was ...|                NULL|                NULL|
|Now here with eve...| I’m as afraid as...| worthless questi...|
|Please Please Ple...|                NULL|                NULL|
|I’m so scared I m...| I’ve been shaped...|             suicide|
|                  11|Honetly idkI dont...|                NULL|
|I would do someth...| an

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when

# Initialize Spark session
spark = SparkSession.builder.appName("JupyterSpark").getOrCreate()

# Load your CSV data into a DataFrame
df = spark.read.csv(r'C:\Users\Hp\Desktop\Suicide_Detection.csv', header=True, inferSchema=True)

# Null ve boş değerleri kaldırarak yeni bir DataFrame oluşturun
df_clean = df.filter(df.text.isNotNull())  # Null değerleri kaldırma
df_clean = df_clean.filter(df_clean.text != "")  # Boş metinleri temizleme


# Create a new column for binary labels
df = df.withColumn("label", when(df["class"] == "suicide", 1).otherwise(0))

# Display the updated DataFrame
df.show()

+--------------------+--------------------+--------------------+-----+
|                 _c0|                text|               class|label|
+--------------------+--------------------+--------------------+-----+
|                   2|Ex Wife Threateni...|             suicide|    1|
|                   3|Am I weird I don'...|         non-suicide|    0|
|                   4|"Finally 2020 is ...|         non-suicide|    0|
|                   8|i need helpjust h...|             suicide|    1|
|                   9|I’m so lostHello,...|                NULL|    0|
|I was later trans...|                NULL|                NULL|    0|
|Come to find out ...| my mom has been ...| therapy has help...|    0|
|I eventually was ...|                NULL|                NULL|    0|
|Now here with eve...| I’m as afraid as...| worthless questi...|    0|
|Please Please Ple...|                NULL|                NULL|    0|
|I’m so scared I m...| I’ve been shaped...|             suicide|    1|
|     

In [8]:
# text sütununu string türüne dönüştür
from pyspark.sql.functions import col
df = df.withColumn("text", col("text").cast("string"))

# Veri türünü kontrol et
df.printSchema()


root
 |-- _c0: string (nullable = true)
 |-- text: string (nullable = true)
 |-- class: string (nullable = true)
 |-- label: integer (nullable = false)



In [9]:
from pyspark.ml.feature import Tokenizer, CountVectorizer
from pyspark.sql.functions import col

# Tokenizer kullanarak metni kelimelere ayır
tokenizer = Tokenizer(inputCol="text", outputCol="words")
words_df = tokenizer.transform(df)

# Null değerleri temizle
words_df = words_df.filter(words_df.words.isNotNull())
df_clean = df.filter((col("text").isNotNull()) & (col("text") != ""))

# Kelimelere ayırdıktan sonra veri tipini kontrol edin
words_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- text: string (nullable = true)
 |-- class: string (nullable = true)
 |-- label: integer (nullable = false)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [10]:
from pyspark.ml.feature import Tokenizer, CountVectorizer
from pyspark.sql.functions import col, size

# 1. Adım: Null ve boş değerleri filtreleme (hem metni hem de kelimeleri)
df_clean = df.filter((col("text").isNotNull()) & (col("text") != ""))

# 2. Adım: Tokenizer işlemi
tokenizer = Tokenizer(inputCol="text", outputCol="words")
words_df = tokenizer.transform(df_clean)

# 3. Adım: Kelimelerin boş olmadığı ve null olmadığı satırları filtreleme
words_df = words_df.filter((col("words").isNotNull()) & (size(col("words")) > 0))

# 4. Adım: CountVectorizer ile kelimeleri vektöre dönüştürme
count_vectorizer = CountVectorizer(inputCol="words", outputCol="features")
model = count_vectorizer.fit(words_df)
vectorized_df = model.transform(words_df)

# 5. Adım: Vektörize edilmiş veriyi gösterme
vectorized_df.show(truncate=False)

# 6. Adım: Veri setini eğitim ve test verilerine bölelim
train_data, test_data = vectorized_df.randomSplit([0.8, 0.2], seed=1234)

# Eğitim ve test verilerini göster
train_data.show(truncate=False)
test_data.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Spark session başlatma
spark = SparkSession.builder.appName("LogisticRegressionExample").getOrCreate()

# Veriyi yükleme (örneğin bir CSV dosyasını okuma)
# vectorized_df = spark.read.csv("path_to_your_data.csv", header=True, inferSchema=True)

# 'label' sütununu sayısal değerlere dönüştür
indexer = StringIndexer(inputCol="label", outputCol="indexedLabel")

# 'indexed_df' adlı DataFrame, 'StringIndexer' ile dönüştürülmüş veriyi tutacaktır.
indexed_df = indexer.fit(vectorized_df).transform(vectorized_df)

# Veriyi eğitim ve test verisine ayır
train_data, test_data = indexed_df.randomSplit([0.8, 0.2], seed=1234)

# Lojistik regresyon modelini başlatıyoruz
lr = LogisticRegression(featuresCol="features", labelCol="indexedLabel", maxIter=10, regParam=0.01, elasticNetParam=0.8)

# Modeli eğitim verisi ile eğit
model = lr.fit(train_data)

# Test verisi üzerinde tahminler yap
predictions = model.transform(test_data)

# Modelin değerlendirilmesi
evaluator = BinaryClassificationEvaluator(labelCol="indexedLabel", rawPredictionCol="prediction")
accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})  # AUC metriğini kullanıyoruz

# Sonuçları yazdır
print(f"Test Accuracy (AUC): {accuracy}")

Test Accuracy (AUC): 0.6159241251173388


In [12]:
from pyspark.sql.functions import col
train_data = train_data.withColumn("label", col("label").cast("double"))
test_data = test_data.withColumn("label", col("label").cast("double"))

In [13]:
print(train_data.printSchema())

root
 |-- _c0: string (nullable = true)
 |-- text: string (nullable = true)
 |-- class: string (nullable = true)
 |-- label: double (nullable = false)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- indexedLabel: double (nullable = false)

None


In [14]:
train_data.filter(train_data["label"].isNull()).show()
test_data.filter(test_data["label"].isNull()).show()
train_data = train_data.dropna()
test_data = test_data.dropna()

+---+----+-----+-----+-----+--------+------------+
|_c0|text|class|label|words|features|indexedLabel|
+---+----+-----+-----+-----+--------+------------+
+---+----+-----+-----+-----+--------+------------+

+---+----+-----+-----+-----+--------+------------+
|_c0|text|class|label|words|features|indexedLabel|
+---+----+-----+-----+-----+--------+------------+
+---+----+-----+-----+-----+--------+------------+



In [15]:
from kafka import KafkaProducer
import pandas as pd
import json
import time

# Create a Kafka producer with the specified API version
kafka_producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    client_id="python-test-producer",
    
    acks=0,  # Ensures that the message is acknowledged by all in-sync replicas (stronger durability guarantee)
    retries=3,  # Retry sending if the message fails
    request_timeout_ms=3000,  # Timeout for a request in milliseconds (adjust as needed)
    metadata_max_age_ms=300000,
    
    api_version=(2, 4, 0),  # Specify the API version as a tuple (major, minor, patch)
    value_serializer=lambda v: json.dumps(v).encode('utf-8')  # Serialize JSON messages
)

# CSV dosyasını oku
data = pd.read_csv('Suicide_Detection.csv')

# Verileri Kafka'ya sırayla gönder
for index, row in data.head(20).iterrows():
    item = row.to_dict()  # Satırı sözlük olarak al
    try:
        kafka_producer.send('my-topic', value=item)
        print(f'Sent: {item}')
    except Exception as e:
        print(f"Error sending message: {e}")
    
    time.sleep(1)  # Her bir gönderim arasında 1 saniye bekle (isteğe bağlı)

kafka_producer.flush()
# Üreticiyi kapat
kafka_producer.close()


Sent: {'Unnamed: 0': 2, 'text': "Ex Wife Threatening SuicideRecently I left my wife for good because she has cheated on me twice and lied to me so much that I have decided to refuse to go back to her. As of a few days ago, she began threatening suicide. I have tirelessly spent these paat few days talking her out of it and she keeps hesitating because she wants to believe I'll come back. I know a lot of people will threaten this in order to get their way, but what happens if she really does? What do I do and how am I supposed to handle her death on my hands? I still love my wife but I cannot deal with getting cheated on again and constantly feeling insecure. I'm worried today may be the day she does it and I hope so much it doesn't happen.", 'class': 'suicide'}
Sent: {'Unnamed: 0': 3, 'text': "Am I weird I don't get affected by compliments if it's coming from someone I know irl but I feel really good when internet strangers do it", 'class': 'non-suicide'}
Sent: {'Unnamed: 0': 4, 'text':

In [ ]:
from kafka import KafkaConsumer
import json

# Create a Kafka consumer
consumer = KafkaConsumer(
    'my-topic',
    bootstrap_servers='localhost:9092',  # Adjust if your Kafka server is different
    value_deserializer=lambda x: json.loads(x.decode('utf-8')),  # Deserialize messages
    api_version=(2, 4, 0)
)

# Consume messages
for message in consumer:
    print(f"Received message: {message.value}")

# Close the consumer (you can also stop it with a break condition)
consumer.close()


In [ ]:
spark.stop()
